In [4]:
from bs4 import BeautifulSoup
import io
import tiktoken

In [6]:
def extract_text(filename):
    f = io.open(filename, mode="r", encoding="utf-8")
    html_doc = f.read()
    f.close()
    soup = BeautifulSoup(html_doc, 'html.parser')
    text = soup.get_text()
    return text

In [ ]:
def chunking(text, max_token_count):
    enc = tiktoken.get_encoding("cl100k_base")

    words = text.split(" ")
    chunk = []
    encodeStr = ""
    token_count = 0
    
    
    for word in words:
        if token_count+len(enc.encode(word))>max_token_count:
            chunk.append(enc.encode(encodeStr))
            encodeStr=""
            token_count = 0
        else:
            encodeStr = encodeStr + word
            token_count += len(enc.encode(word))
            
    chunk.append(enc.encode(encodeStr))
        
    return chunk

In [ ]:
def extract_and_chunk_text(filename, max_token_count):
    text = extract_text(filename)
    
    chunks = chunking(text, max_token_count)
    
    return chunks